<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/notebooks/ALE_Framework_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if 'google.colab' in sys.modules:
    !rm -rf *
    !git clone https://github.com/Tinynja/Sarsa-phi-EB
    !mv Sarsa-phi-EB/* .
    !rm -rf Sarsa-phi-EB
    # DON'T install packages defined in Pipfile_Colab_exclude
    !sed -ri "/$(tr '\n' '|' < Pipfile_Colab_exclude)/d" Pipfile
else:
    print('Skipping GitHub cloning since not running in Colab.')

Cloning into 'Sarsa-phi-EB'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (285/285), done.
remote: Total 336 (delta 112), reused 204 (delta 39), pack-reused 0
Receiving objects: 100% (336/336), 798.62 KiB | 10.94 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [2]:
# Install required dependencies
import os

if 'google.colab' in sys.modules:
    # Some dependencies required for displaying episode
    !apt install -y python-opengl xvfb
    !pip install pyvirtualdisplay
    # Colab doesn't support pipenv, hence we convert Pipfile into requirements.txt
    if 'requirements_Colab.txt' not in os.listdir():
        !pip install pipenv
        !pipenv lock -r > requirements.txt
    !pip install -r requirements_Colab.txt 1> /dev/null
else:
    !pipenv install 1> /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl xvfb
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 1,281 kB of archives.
After this operation, 7,686 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 1,281 kB in 1s (1,404 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Selecting previously unselected package xvfb.
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Setting

In [3]:
# Import all supported ROMs into ALE
!ale-import-roms ROMS

[SUPPORTED]           kung_fu_master ROMS/Kung-Fu Master (1987).bin
[SUPPORTED]           tic_tac_toe_3d ROMS/3-D Tic-Tac-Toe (1980).bin
[SUPPORTED]              road_runner    ROMS/Road Runner (1989).bin
[SUPPORTED]                ms_pacman    ROMS/Ms. Pac-Man (1983).bin
[SUPPORTED]               beam_rider      ROMS/Beamrider (1984).bin
[SUPPORTED]                    qbert        ROMS/Q. Bert (1983).bin
[SUPPORTED]                atlantis2    ROMS/Atlantis II (1982).bin
[SUPPORTED]                 trondead ROMS/TRON - Deadly Discs (TRON Joystick) (1983).bin
[SUPPORTED]                 superman       ROMS/Superman (1979).bin
[SUPPORTED]             flag_capture ROMS/Flag Capture - Capture (Capture the Flag) (1978).bin
[SUPPORTED]                blackjack ROMS/Blackjack - Black Jack (Gambling) (Paddle) (1977).bin
[SUPPORTED]                    qbert         ROMS/Q-bert (1983).bin
[SUPPORTED]                 seaquest       ROMS/Seaquest (1983).bin
[SUPPORTED]                   skiing RO

In [4]:
#### ALE-related imports ####

# Built-in libraries
import re
import sys
import base64
import pickle
import random
import subprocess
from pathlib import Path
from datetime import datetime

# Pypi libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from ale_py import ALEInterface, SDL_SUPPORT
import ale_py.roms as ROMS

# Episode display
from PIL import Image
from pyvirtualdisplay import Display
Display().start()

# Configuration
CUDA = 'cuda' if torch.cuda.device_count() else 'cpu'
CPU = 'cpu'

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [5]:
class EnvRecorder:
    def __init__(self, env, out_dir='./results'):
        self.out_dir = Path(out_dir)
        self.out_dir.mkdir(exist_ok=True)
        self.out_dir = self.out_dir.resolve()
        self.env = env

        # Workspace variables
        self.__frame_count_padding = 0
        self.__timestep = 0
    
    def __call__(self, choose_action=None, max_steps=-1, height=300):
        self.record_episode_and_show(choose_action=choose_action, max_steps=max_steps, height=height)
    
    def record_episode_and_show(self, choose_action=None, max_steps=-1, height=300):
        self.record_episode(choose_action=choose_action, max_steps=max_steps)
        self.show_recording(height=height)
    
    def record_episode(self, choose_action=None, max_steps=-1):
        """Generate an episode and record it as record.mp4
        Args:
        choose_action -- callable method that returns the next action based on the current observation
        max_steps     -- maximum number of steps after which force end episode
        """
        choose_action = choose_action or (lambda _: self.env.action_space.sample())

        self._clear_recording()
        self.__timestep, done, observation =  0, False, self.env.reset()
        self._record_frame()
        while not done and self.__timestep != max_steps:
            print(f'\rRecording episode, timestep {self.__timestep+1}...', end='')
            action = choose_action(observation)
            observation, _, done, _ = self.env.step(action)
            self.__timestep += 1
            self._record_frame()

        if self.__timestep == max_steps and not done:
            print('\nWarning: `max_steps` reached before episode terminated')
        else:
            print()

        self._export_as_mp4()

    def show_recording(self, height=300):
        """Show a .mp4 video in html format of the recorded episode"""
        filepath = self.out_dir.joinpath('record.mp4')
        video_b64 = base64.b64encode(filepath.read_bytes())
        html = f'''<video alt="{filepath}" autoplay loop controls style="height:{height}px">
                        <source src="data:video/mp4;base64,{video_b64.decode('ascii')}" type="video/mp4" />
                   </video>'''
        ipythondisplay.display(ipythondisplay.HTML(data=html))

    def _clear_recording(self):
        # This is a new episode, delete previously recorded steps
        self.out_dir.joinpath('record').mkdir(exist_ok=True)
        for step_png in self.out_dir.glob('record/step_*.png'):
            step_png.unlink()
        if self.out_dir.joinpath('record.mp4').exists():
            self.out_dir.joinpath('record.mp4').unlink()
    
    def _record_frame(self):
        # Record current timestep png
        img = Image.fromarray(env.render('rgb_array'))
        out_path = self.out_dir.joinpath(f'record/step_{self.__timestep}.png')
        img.save(str(out_path))

    def _export_as_mp4(self):
        """Convert the recorded set of png files into an mp4 video"""
        self._standardize_frame_count_padding()
        in_dir = self.out_dir.joinpath('record')
        in_pattern = f'step_%0{self.__frame_count_padding}d.png'
        out_file = self.out_dir.joinpath('record.mp4')
        !cd $in_dir; ffmpeg -hide_banner -loglevel error -r 60 -i $in_pattern -vcodec libx264 -crf 25 -pix_fmt yuv420p -y $out_file
    
    def _standardize_frame_count_padding(self):
        self.__frame_count_padding = len(str(self.__timestep))
        number_pattern = re.compile('\d+')
        png_abs_glob = 'step_*.png'
        for png_path in self.out_dir.joinpath('record').glob(png_abs_glob):
            ts = int(number_pattern.search(png_path.stem).group(0))
            new_name = png_path.parent.joinpath(f'step_{ts:0{self.__frame_count_padding}d}.png')
            png_path.rename(new_name)

In [6]:
class features:
    @staticmethod
    def basic(frame, palette, background, crop_size=torch.Tensor([15,10]), flatten=True):
        # For each color in palette, tell if each pixel is that color (e.g. 210,160,128)
        colors_in_pixels = ((frame-background).unsqueeze(-2) == palette).all(-1)
        # Split the image into n tiles, each with dimension `crop_size` (e.g. 14,16,15,10,128)
        cropped_colors_in_pixels = torch.stack(torch.stack(colors_in_pixels.split(crop_size[1],dim=-2)).split(crop_size[0],dim=-3))
        # Apply logical or inside each cropped image (e.g. 14,16,128)
        cropped_features = cropped_colors_in_pixels.any(3).any(2)
        # Flatten the features (e.g. 28672)
        return cropped_features.flatten() if flatten else cropped_features
    
    @staticmethod
    def b_pros(frame, palette, background, crop_size=torch.Tensor([15,10])):
        raise NotImplementedError()
        basic_features = features.basic(frame, palette, background, crop_size=crop_size)
        b_pros_features = torch.combinations(basic_features)
        return b_pros_features

In [ ]:
from gym.spaces import Discrete

class EnvALE:
    def __init__(self, rom, out_dir='results', seed=0, feature_type='ScreenRGB',
                 regen_bg=False, bg_samples=18000, device=CUDA):
        self.rom = rom
        self.rom_name = rom.stem
        self.feature_type = feature_type
        self.device = device

        self.out_dir = Path(out_dir)
        self.out_dir.mkdir(exist_ok=True)
        self.out_dir = self.out_dir.resolve()

        # ALE
        self.ale = ALEInterface()
        self.ale.setInt("random_seed", seed)
        self.ale.loadROM(rom)

        # gym action_space compatibility
        action_set = self.ale.getMinimalActionSet()
        self.action_space = Discrete(len(action_set))
        self.action_space.action_set = action_set

        # color palette
        self.color_palette = self._get_color_palette().to(self.device)

        self._bg_path = Path(f'./backgrounds/{self.rom_name}.pickle')
        if regen_bg or not self._bg_path.exists() or not self._bg_path.is_file():
            self.background = self._get_background(n_samples=bg_samples)
        else:
            with open(self._bg_path, 'rb') as file:
                self.background = pickle.load(file).to(self.device)
        
        self._set_observe_method(feature_type)
        self.observation_space = self._observe().shape

        # Default values
        self._timestep = 0
        self._do_record = False
        self._record_padding = None

    def reset(self, do_record=False):
        self.ale.reset_game()
        observation = self._observe()
        self._timestep = 0
        return observation
        
    def step(self, action, repeat=1):
        if not isinstance(repeat, int) or repeat <= 0: raise ValueError(f'`repeat` must be an integer greater than 0 (got `{repeat}`)')

        if isinstance(action, int):
            action = self.action_space.action_set[action]

        done, sub_timestep, reward = False, 0, 0
        while sub_timestep < repeat and not done:
            reward += self.ale.act(action)
            self._timestep += 1
            sub_timestep += 1
            done = self.ale.game_over()
        observation = self._observe()
                
        return observation, reward, done, None

    def render(self, mode='rgb_array'):
        if mode == 'rgb_array':
            return self.ale.getScreenRGB()
        else:
            raise ValueError(f'render mode `{mode}` is not supported')

    def _set_observe_method(self, feature_type):
        if feature_type == 'ScreenRGB':
            self._observe = lambda: torch.from_numpy(self.ale.getScreenRGB()).to(self.device)
        elif feature_type == 'ScreenGrayscale':
            self._observe = lambda: torch.from_numpy(self.ale.getScreenGrayscale()).to(self.device)
        elif feature_type == 'Basic':
            self._observe = lambda: features.basic(frame=torch.from_numpy(self.ale.getScreenRGB()).to(self.device),
                                                   palette=self.color_palette,
                                                   background=self.background)
        elif feature_type == 'B-PROS':
            self._observe = lambda: features.b_pros(frame=torch.from_numpy(self.ale.getScreenRGB()).to(self.device),
                                                    palette=self.color_palette,
                                                    background=self.background)
        else:
            raise NotImplementedError(f'Feature type `{feature_type}` is not supported')
        
    def _observe(self):
        raise NotImplementedError()
    
    def _get_color_palette(self):
        result = subprocess.run(['python', '-c', f'__import__("ale_py").ALEInterface().loadROM("{str(self.rom)}")'], capture_output=True)
        palette_name = result.stderr.decode().splitlines()[6].strip().split()[-1]
        with open(f'palettes/{palette_name}_Palette.pickle', 'rb') as file:
            palette = pickle.load(file)
        return palette
    
    def _get_background(self, n_samples):
        bg_feature_type = 'ScreenRGB' if self.feature_type not in ['ScreenGrayscale',] else 'ScreenGrayscale'
        self._set_observe_method(bg_feature_type)
        
        sample_i = 0
        pixel_histogram = torch.zeros((*self.ale.getScreenDims(), self.color_palette.shape[0]), dtype=int).to(self.device)
        while sample_i < n_samples:
            done, observation = False, self.reset()
            while not done and sample_i < n_samples:
                if not sample_i%10:
                    print(f'\rGenerating background... {sample_i}/{n_samples} samples ({sample_i/n_samples:.0%})', end='')
                action = random.choice(self.action_space)
                observation, reward, done, info = self.step(action)
                observation = torch.from_numpy(observation).to(self.device)
                colors_in_pixels = (observation.unsqueeze(-2) == self.color_palette).all(-1)
                # for i in range(colors_in_pixels.shape[-1]):
                #     print(colors_in_pixels.reshape(-1, 128))
                pixel_histogram += colors_in_pixels
                sample_i += 1
        background_ids = pixel_histogram.argmax(axis=-1)
        background = self.color_palette[background_ids]
        
        self._bg_path.parent.mkdir(exist_ok=True)
        with open(self._bg_path, 'wb') as file:
            pickle.dump(background.cpu(), file)
        
        return background

In [ ]:
#@title

if print("Skipped background regeneration."):
# if True:

    from ale_py.roms import *
    games_to_generate_bg = [Breakout, MontezumaRevenge, Venture, Qbert, Frostbite, Freeway]

    for game in games_to_generate_bg:
        print(game.stem)
        env = EnvALE(game, regen_bg=True)
        plt.imshow(env.background.cpu().numpy())
        plt.show()

In [ ]:
#@title

if print("Skipped displaying stored backgrounds to reduce ouptuts."):
# if True:

    for filepath in Path('backgrounds').iterdir():
        print(f'Background in `{filepath.resolve()}`')
        with open(filepath, 'rb') as file:
            bg = pickle.load(file)
        plt.imshow(bg)
        plt.show()

In [ ]:
#@title

if print("Skipped manual test."):
# if True:

    env = EnvALE(ROMS.Breakout)
    recorder = EnvRecorder(env)
    recorder()

In [10]:
#######################################################################
# Copyright (C)                                                       #
# 2017-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

# import numpy as np
# import matplotlib
# matplotlib.use('Agg')
# import matplotlib.pyplot as plt
# from math import floor
# from tqdm import tqdm


# all possible actions
ACTIONS = range(4)

# discount is always 1.0 in these experiments
DISCOUNT = 0.99

# use optimistic initial value, so it's ok to set epsilon to 0
EPSILON = 0.01

# maximum steps per episode
STEP_LIMIT = 5000


# get action at @position and @velocity based on epsilon greedy policy and @valueFunction  #########################    use our own get_action. modified it, may work as intended
def get_action(observation, valueFunction):
    if np.random.binomial(1, EPSILON) == 1:
        return np.random.choice(ACTIONS)
    values = []
    for action in ACTIONS:
        values.append(valueFunction.value(observation))  
    maxi = np.max(values)
    bestactions = np.where(values==maxi,1.0,0.0)
    for i in range(len(ACTIONS)):
      bestactions[i] = bestactions[i]*np.random.uniform()
    action = np.argmax(bestactions)
    return action



# replacing trace update rule
# @trace: old trace (will be modified)
# @activeTiles: current active tile indices
# @lam: lambda
# @return: new trace for convenience
def replacing_trace(trace, activeTiles, lam):
    active = (torch.arange(len(trace)).to(device)[None,...] == activeTiles.flatten()[...,None]).any(0)
    trace[active] = 1
    trace[~active] *= lam * DISCOUNT
    return trace



# wrapper class for Sarsa(lambda)
class Sarsa:
    # In this example I use the tiling software instead of implementing standard tiling by myself
    # One important thing is that tiling is only a map from (state, action) to a series of indices
    # It doesn't matter whether the indices have meaning, only if this map satisfy some property
    # View the following webpage for more information
    # http://incompleteideas.net/sutton/tiles/tiles3.html
    # @maxSize: the maximum # of indices
    #the hashing is a lfa?
    def __init__(self, step_size, lam, trace_update=replacing_trace, max_size=28672, initial_weights=0):
        self.max_size = max_size
        self.trace_update = trace_update
        self.lam = lam

        # divide step size equally to each tiling
        self.step_size = step_size/10

        # weight for each tile
        if initial_weights == 0:
          self.weights =torch.zeros(max_size).to(device) #max size is the number of features?
        else:
          self.weights = initial_weights.to(device)

        # trace for each tile
        self.trace = torch.zeros(max_size).to(device)



    # estimate the value of given state and action
    def value(self, observation):
        active_tiles = np.nonzero(observation)
        return self.weights[active_tiles].sum()

    # learn with given state, action and target
    def learn(self, observation, target):
        active_tiles = np.nonzero(observation)
        estimation = self.weights[active_tiles].sum()
        delta = target - estimation
        #print('estimation array: ' + str(self.weights[active_tiles]))
        # print('estimation: ' + str(self.weights[active_tiles].sum()))
        if self.trace_update == replacing_trace:
            self.trace_update(self.trace, active_tiles, self.lam)
        else:
            raise Exception('Unexpected Trace Type')
        self.weights += self.step_size * delta * self.trace
        # print('delta: ' + str(delta))
        # print('weights: ' +  str(self.weights))


# play Mountain Car for one episode based on given method `evaluator`
# return: total steps in this episode
def play(evaluator, env):

    action = random.choice(ACTIONS)
    steps = 0
    while True:
        next_observation, reward, done, info = env.step(action)
        next_action = get_action(next_observation, evaluator)    #########################    use our own get_action  ??? modified it, may work as intented
        steps += 1
        target = reward + DISCOUNT * evaluator.value(next_observation)          ############# use our own value function ??? modified it, may work as intented
        evaluator.learn(observation, target)
        observation = next_observation
        action = next_action
        if done:
            break
        if steps >= STEP_LIMIT:
            print('Step Limit Exceeded!')
            break
    return steps

class BaseAgent:
  """ The base agent class function.
  """
  def __init__(self, nb_features=28672):
    #nothing for now
    self.gamma = 1
    self.features = nb_features
    self.rhos = torch.ones(self.features).to(device) #stores the rho_i values


  def takeAction(self, t):
    phis = [[0,1,0],[0,1,0],[0,1,0],[1,0,1]]
    return phis[t]


  def updateRho_i(self, counts, t):
    M = self.features
    self.rhos = (counts+1.5)/(t+1)
    return 0


  def PHI_EB(self, evaluator, env, beta=0.05, t_end=200):
    t = 0
    M = self.features #number of features
    counts = torch.zeros(M).to(device)
    states = torch.zeros(t_end,M).to(device) #stores the previous phis for all timesteps

    action = 1 #starting the game for the agent on the first game
    old_phi = env._observe()
    print('starting iterations')
    print('rhos: ' + str(self.rhos))
    while t < t_end:
    #   print("Iteration #{}".format(t))
      #observe phi(s), reward
    #   phi, reward, done, info = obs.clone(), 0, False, None
      phi, reward, done, info = env.step(action)
    #   print(phi.shape)
    #   print('--------------------------------------------------------------')
    #   print('took action: ', env.action_space[action])
      next_action = get_action(phi, evaluator)
    #   print('phi: ' + str(phi))
      
      #compute rho_t(phi) (feature visit-density)
      if t > 0:
        counts = (phi==states[0:t]).sum(0)
        # print(counts)
        self.rhos = (counts+0.5)/(t+1)
        # print('rhos: ' + str(self.rhos))
        rho_t = torch.prod(self.rhos)
      else:
        rho_t = 0.5**M
    #   print('rho_t ' + str(rho_t))
      #update all rho_i with observed phi
      states[t] = phi
      self.updateRho_i(counts, t+1)
    #   print('min rho_i_t: ' + str(min(self.rhos)))
      
      #compute rho_t+1(phi)
      new_rho_t = 1
      # THIS IS A BOTTLENECK (tested in CPU mode: 74ms -> 178ms)
      for i in range(M):
        new_rho_t = new_rho_t*self.rhos[i]
      if new_rho_t <= 1e-323: #this is to avoid division by zero, might need to be tweaked
        new_rho_t = 1e-323
    #   print('new_rho_t ' + str(new_rho_t))

      #compute Nhat_t(s)
      Nhat_t = rho_t*(1-new_rho_t)/(new_rho_t-rho_t)
    #   print('Nhat_t: ',   Nhat_t)
      if Nhat_t <= 1e-323: #this is to avoid division by zero again, might need to be tweaked
        Nhat_t = torch.tensor([1e-323]).to(device)

      #compute R(s,a) (empirical reward)
      explorationBonus = beta/torch.sqrt(Nhat_t)
      if torch.isnan(explorationBonus) or explorationBonus >= 1e3:
        explorationBonus = 1e3

      reward = reward + explorationBonus
    #   print('reward: ',reward)

    #   print('state value: ' + str(evaluator.value(phi)))
      #pass phi(s) and reward to RL algo to update theta_t
      target = reward + self.gamma * evaluator.value(phi)          ############# use our own value function ??? modified it, may work as intented
      # THIS IS A BOTTLENECK (tested in CPU mode: 190ms -> 207ms)
      evaluator.learn(old_phi, target)

      if done:
        #break
        env.reset()
        action = 1
        old_phi = env._observe()
        print('episode ended on step ', t, 'starting a new one')
      else:
        old_phi = phi
        action = next_action
      t += 1
      continue


    return evaluator.weights

# from ale_py.roms import Breakout
# import timeit
# env = EnvALE(Breakout, feature_type='Basic')
# print(env.action_space)
# alpha = 0.5
# lam = 0.9
# #we can upload previous weights as as tensor, or initialize at 0
# previous_weights = 0
# obs = torch.randint(0,2,(28672,), dtype=bool).to(device)
# t_end = 10

# evaluator = Sarsa(alpha, lam, replacing_trace, 28672, previous_weights)
# agent = BaseAgent()
# env.reset(do_record=False)
# start_time = timeit.default_timer()
# weights = agent.PHI_EB(evaluator, env, beta=0.05, t_end=t_end)
# print(f'{1000*(timeit.default_timer()-start_time)/t_end}ms per timestep')










import timeit
import logging

import torch
import numpy

class SarsaPhiEBAgent:
    def __init__(self, env, epsilon=0.01, beta=0.05, init_action=None, step_repeat=4, device=CUDA, debug=False):
        """An agent using Sarsa(lambda) algorithm with:
            - Linear Function Approximation (SGD)
            - Replacing Traces
            - Exploration-Bonus
        """
        self.env = env
        self.init_action = init_action
        self.epsilon = epsilon
        self.beta = beta
        self.step_repeat = step_repeat
        self.device = device

        self.__log = logging.Logger(name=self.__class__.__name__,
                                    level=(logging.DEBUG if debug else logging.WARNING))

        # TO REWORK
        self.evaluator = Sarsa(step_size=0.5, lam=0.9, trace_update=replacing_trace, max_size=28672, initial_weights=0)
        self.gamma = 1

        # Init some workspace variables
        self.weights = torch.zeros(self.env.observation_space.shape).to(self.device)
        self._counts = torch.zeros(self.env.observation_space.shape).to(self.device)
        self._total_steps = 0
        self._done = True
    
    def _env_reset(self, do_record=False):
        observation = self.env.reset(do_record=do_record)
        if self.init_action:
            observation = self.env.step(self.init_action)
        return observation
    
    def _env_step_continuous(self, action):
        if self._done:
            self._env_reset()
        observation, reward, self._done, info = self.env.step(action, repeat=self.step_repeat)
        return observation, reward
    
    def choose_action(self, phi):
        if torch.rand(1) > self.epsilon:
            action = self.weights@phi
        else:
            action = torch.randint(self.env.action_space.n, (1,))
        return action.item()
    
    def visualize_episode(self):
        # Play an episode
        total_reward, done, phi = 0, False, self._env_reset(do_record=True)
        while not done:
            old_phi = phi # TODO: REWORK
            action = get_action(old_phi, self.evaluator) # TODO: REWORK
            phi, reward, done, info = self.env.step(action)
            total_reward += reward
            print(f'\rTimestep {env._timestep}...', end='')
        print('\n\r' + f'Total reward: {total_reward}')

        # Show episode
        self.env.show_video()

    def learn(self, n_steps):
        phi = self._env_reset()
        start_time = timeit.default_timer()
        for i in range(n_steps):
            # Information display
            avg_time = (timeit.default_timer()-start_time)/(i or 1)
            print(f'\rSarsaPhiEB learning iteration {self._total_steps+1}/{self._total_steps-i+n_steps} ({1000*avg_time:.2f}ms/iter, remaining {(n_steps-i)*avg_time:.0f}s)    ', end='')

            # Take an action
            old_phi = phi.cpu() # TODO: REWORK
            action = self.choose_action(old_phi) # TODO: REWORK
            phi, reward = self._env_step_continuous(action)
            self._total_steps += 1
            
            # Compute the exploration bonus
            phi_occ = torch.cat((self._counts[phi], self._total_steps-self._counts[~phi])).to(self.device)
            rho = ((phi_occ+1/2) / (self._total_steps+1)).prod()
            rho_prime = ((phi_occ+1+1/2) / (self._total_steps+1+1)).prod()
            pseudocount = (rho*(1-rho_prime)) / (rho_prime-rho)
            exploration_bonus = self.beta/pseudocount.sqrt()

            reward_augmented = reward + exploration_bonus

            # RL Algorithm
            # TODO: REWORK
            target = reward + self.gamma * self.evaluator.value(phi) ############# use our own value function ??? modified it, may work as intented
            self.evaluator.learn(old_phi, target)
            self.weights = self.evaluator.weights
        print(f'\nTotal elapsed time: {datetime.utcfromtimestamp(timeit.default_timer()-start_time).strftime("%H:%M:%S.%f")}')

device = 'cpu'
env = EnvALE(ROMS.Breakout, feature_type='Basic', device=CUDA)

import gym
env = gym.make('CartPole-v1')
agent = SarsaPhiEBAgent(env)

In [39]:
# !apt-get install python-opengl -y

# !apt install xvfb -y

# !pip install pyvirtualdisplay

# !pip install piglet


from pyvirtualdisplay import Display
Display().start()

import gym
from IPython import display
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

# env = gym.make('CartPole-v0')
# env.reset()
# img = Image.fromarray(env.render('rgb_array')) # only call this once
# img.save('frames/0.png')
# for i in range(40):
#     # display.display(plt.gcf())
#     # display.clear_output(wait=True)
#     action = env.action_space.sample()
#     env.step(action)
#     img = Image.fromarray(env.render('rgb_array')) # just update the data
#     img.save(f'frames/{i+1}.png')




env = gym.make('MountainCar-v0')
recorder = EnvRecorder(env)
recorder()

Recording episode, timestep 200...


In [33]:
!cd results/record; ffmpeg -hide_banner -loglevel error -r 60 -i step_%02d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p -y record.mp4

In [26]:
env.reset()
env.render('rgb_array').shape

(400, 600, 3)

In [11]:
agent.visualize_episode()

TypeError: ignored

In [ ]:
agent.learn(99000)
agent.visualize_episode()

In [ ]:

plt.imshow(prev_screen)

for i in range(50):
  action = env.action_space.sample()
  obs, reward, done, info = env.step(action)
  screen = env.render(mode='rgb_array')

  plt.imshow(screen)
  ipythondisplay.clear_output(wait=True)
  ipythondisplay.display(plt.gcf())

  if done:
    break

ipythondisplay.clear_output(wait=True)
env.close()